In [1]:
# importing essential libraries
import pandas as pd
import numpy as np
import os
import warnings
from pandasql import sqldf
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# supressing warning messages
warnings.filterwarnings('ignore')

In [3]:
# locating the data
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
data_path = os.path.join(parent_dir,"data","test.csv")

#loading the data
df = pd.read_csv(data_path)
df.head()

,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
0,1.311448e+19,4/4/2019 12:01,770.0,4/25/2019 14:35,662.0,1823652.0,2.082014e+14,3.366496e+10,3.552121e+13,9.16457E+15,...,15854611.0,2501332.0,8198936.0,9656251.0,278082303.0,14344150.0,171744450.0,8814393.0,36749741.0,308879636.0
1,1.311448e+19,4/9/2019 13:04,235.0,4/25/2019 8:15,606.0,1365104.0,2.082019e+14,3.368185e+10,3.579401e+13,L77566A,...,20247395.0,19111729.0,18338413.0,17227132.0,608750074.0,1170709.0,526904238.0,15055145.0,53800391.0,653384965.0
2,1.311448e+19,4/9/2019 17:42,1.0,4/25/2019 11:58,652.0,1361762.0,2.082003e+14,3.376063e+10,3.528151e+13,D42335A,...,19725661.0,14699576.0,17587794.0,6163408.0,229584621.0,395630.0,410692588.0,4215763.0,27883638.0,279807335.0
3,1.311448e+19,4/10/2019 0:31,486.0,4/25/2019 7:36,171.0,1321509.0,2.082014e+14,3.375034e+10,3.535661e+13,T21824A,...,21388122.0,15146643.0,13994646.0,1097942.0,799538153.0,10849722.0,749039933.0,12797283.0,43324218.0,846028530.0
4,1.311448e+19,4/12/2019 20:10,565.0,4/25/2019 10:40,954.0,1089009.0,2.082014e+14,3.369980e+10,3.540701e+13,D88865A,...,15259380.0,18962873.0,17124581.0,415218.0,527707248.0,3529801.0,550709500.0,13910322.0,38542814.0,569138589.0


In [4]:
# looking at available features
df.columns.to_list()

['Bearer Id',
 'Start',
 'Start ms',
 'End',
 'End ms',
 'Dur. (ms)',
 'IMSI',
 'MSISDN/Number',
 'IMEI',
 'Last Location Name',
 'Avg RTT DL (ms)',
 'Avg RTT UL (ms)',
 'Avg Bearer TP DL (kbps)',
 'Avg Bearer TP UL (kbps)',
 'TCP DL Retrans. Vol (Bytes)',
 'TCP UL Retrans. Vol (Bytes)',
 'DL TP < 50 Kbps (%)',
 '50 Kbps < DL TP < 250 Kbps (%)',
 '250 Kbps < DL TP < 1 Mbps (%)',
 'DL TP > 1 Mbps (%)',
 'UL TP < 10 Kbps (%)',
 '10 Kbps < UL TP < 50 Kbps (%)',
 '50 Kbps < UL TP < 300 Kbps (%)',
 'UL TP > 300 Kbps (%)',
 'HTTP DL (Bytes)',
 'HTTP UL (Bytes)',
 'Activity Duration DL (ms)',
 'Activity Duration UL (ms)',
 'Dur. (ms).1',
 'Handset Manufacturer',
 'Handset Type',
 'Nb of sec with 125000B < Vol DL',
 'Nb of sec with 1250B < Vol UL < 6250B',
 'Nb of sec with 31250B < Vol DL < 125000B',
 'Nb of sec with 37500B < Vol UL',
 'Nb of sec with 6250B < Vol DL < 31250B',
 'Nb of sec with 6250B < Vol UL < 37500B',
 'Nb of sec with Vol DL < 6250B',
 'Nb of sec with Vol UL < 1250B',
 'Socia

In [5]:
# calculating percentages of missing values in each column 
def calculateMissing(df):
    df2 = df.isna().sum().to_frame().reset_index()
    df2.rename(columns = {'index':'variables', 0:'count'}, inplace = True)
    df2['percent'] = round(df2['count']*100/df.shape[0])
    data_type_lis = df.dtypes.to_frame().reset_index()
    df2['data_type'] = data_type_lis.iloc[:,1]
    
    unique_val = []
    for i in range(df2.shape[0]):
        unique_val.append(len(pd.unique(df[df2.iloc[i,0]])))
    df2['unique_values'] = pd.Series(unique_val)
    return df2

calculateMissing(df)

,variables,count,percent,data_type,unique_values
0,Bearer Id,991,1.0,float64,134709
1,Start,1,0.0,object,9998
2,Start ms,1,0.0,float64,1001
3,End,1,0.0,object,6404
4,End ms,1,0.0,float64,1001
5,Dur. (ms),1,0.0,float64,89526
6,IMSI,570,0.0,float64,107266
7,MSISDN/Number,1066,1.0,float64,106857
8,IMEI,572,0.0,float64,107271
9,Last Location Name,1153,1.0,object,45037


In [6]:
'rdx_session','start','start_ms','end','end_ms','rdx_dur','sim_id','phone_number','device_id','last_location','avg_rtt_dl','avg_rtt_ul','avg_bearer_tp_dl','avg_bearer_tp_ul','tcp_dl_retrans_(Bytes)','tcp_ul_retrans_(Bytes)','DL TP < 50 Kbps (%)','50 Kbps < DL TP < 250 Kbps (%)','250 Kbps < DL TP < 1 Mbps (%)','DL TP > 1 Mbps (%)','UL TP < 10 Kbps (%)','10 Kbps < UL TP < 50 Kbps (%)','50 Kbps < UL TP < 300 Kbps (%)','UL TP > 300 Kbps (%)','http_dl_(Bytes)','http_ul_(Bytes)','activity_dur_dl','activity_dur_ul','Dur. (ms).1','handset_manu','handset_type','Nb of sec with 125000B < Vol DL','Nb of sec with 1250B < Vol UL < 6250B','Nb of sec with 31250B < Vol DL < 125000B','Nb of sec with 37500B < Vol UL','Nb of sec with 6250B < Vol DL < 31250B','Nb of sec with 6250B < Vol UL < 37500B','Nb of sec with Vol DL < 6250B','Nb of sec with Vol UL < 1250B','social_dl_(Bytes)','social_ul_(Bytes)','google_dl_(Bytes)','google_ul_(Bytes)','email_dl_(Bytes)','email_ul_(Bytes)','youtube_dl_(Bytes)','youtube_ul_(Bytes)','netflix_dl_(Bytes)','netflix_ul_(Bytes)','gaming_dl_(Bytes)','gaming_ul_(Bytes)','other_dl_(Bytes)','other_ul_(Bytes)','total_ul_(Bytes)','total_dl_(Bytes)'

('rdx_session',
 'start',
 'start_ms',
 'end',
 'end_ms',
 'rdx_dur',
 'sim_id',
 'phone_number',
 'device_id',
 'last_location',
 'avg_rtt_dl',
 'avg_rtt_ul',
 'avg_bearer_tp_dl',
 'avg_bearer_tp_ul',
 'tcp_dl_retrans_(Bytes)',
 'tcp_ul_retrans_(Bytes)',
 'DL TP < 50 Kbps (%)',
 '50 Kbps < DL TP < 250 Kbps (%)',
 '250 Kbps < DL TP < 1 Mbps (%)',
 'DL TP > 1 Mbps (%)',
 'UL TP < 10 Kbps (%)',
 '10 Kbps < UL TP < 50 Kbps (%)',
 '50 Kbps < UL TP < 300 Kbps (%)',
 'UL TP > 300 Kbps (%)',
 'http_dl_(Bytes)',
 'http_ul_(Bytes)',
 'activity_dur_dl',
 'activity_dur_ul',
 'Dur. (ms).1',
 'handset_manu',
 'handset_type',
 'Nb of sec with 125000B < Vol DL',
 'Nb of sec with 1250B < Vol UL < 6250B',
 'Nb of sec with 31250B < Vol DL < 125000B',
 'Nb of sec with 37500B < Vol UL',
 'Nb of sec with 6250B < Vol DL < 31250B',
 'Nb of sec with 6250B < Vol UL < 37500B',
 'Nb of sec with Vol DL < 6250B',
 'Nb of sec with Vol UL < 1250B',
 'social_dl_(Bytes)',
 'social_ul_(Bytes)',
 'google_dl_(Bytes)',
 '

In [7]:
#selecting some useful columns
drop_lis = ['start','start_ms','end','end_ms','DL TP < 50 Kbps (%)','50 Kbps < DL TP < 250 Kbps (%)','250 Kbps < DL TP < 1 Mbps (%)','DL TP > 1 Mbps (%)','UL TP < 10 Kbps (%)','10 Kbps < UL TP < 50 Kbps (%)','50 Kbps < UL TP < 300 Kbps (%)','UL TP > 300 Kbps (%)''Nb of sec with 125000B < Vol DL','Nb of sec with 1250B < Vol UL < 6250B','Nb of sec with 31250B < Vol DL < 125000B','Nb of sec with 37500B < Vol UL','Nb of sec with 6250B < Vol DL < 31250B','Nb of sec with 6250B < Vol UL < 37500B','Nb of sec with Vol DL < 6250B','Nb of sec with Vol UL < 1250B']
df_s = df[df.columns[~df.columns.isin(drop_lis)]]

In [8]:
#droping columns with more than 30% missing values.
def reduce_dim_missing(df,loca):
    temp = summ_columns(df)
    rem_lis = []
    for i in range(self.temp.shape[0]):
        if(temp.iloc[i,loca]):
            rem_lis.append(temp.iloc[i,loca])
    r_df = df.drop(df.columns[rem_lis], axis=1) 
    
    return r_df
calculateMissing(df3)

NameError: name 'df3' is not defined

In [ ]:

#filling missing values of categorical variables with mode 
def fill_missing_by_mode(df, cols):
    mod_fill_list = []
    for i in range(cols.shape[0]):
        if(cols.iloc[i,3] == "object"):
            mod_fill_list.append(cols.iloc[i,0])
            
    for col in mod_fill_list:
        df[col] = df[col].fillna(df[col].mode()[0])
    return df

df3 = fill_missing_by_mode(df3, calculateMissing(df3))

calculateMissing(df3)

In [ ]:
def fix_missing_ffill(df, cols):
    for col in cols:
        df[col] = df[col].fillna(method='ffill')
    return df

def fill_missing_by_median(df, cols):
    median_fill_list = []

    for i in range(cols.shape[0]):
        if(cols.iloc[i,3] == "float64"):
            median_fill_list.append(cols.iloc[i,0])
            
    for col in median_fill_list:
        df[col] = df[col].fillna(df[col].median())
    return df

def fill_missing_by_mean(df, cols):
    mean_fill_list = []
    for i in range(cols.shape[0]):
        if(cols.iloc[i,3] == "float64"):
            mean_fill_list.append(cols.iloc[i,0])
    
    for col in mean_fill_list:
        df[col] = df[col].fillna(df[col].median())
    return df

df3 = fill_missing_by_median(df3, calculateMissing(df3))
                  


calculateMissing(df3)

In [ ]:
# removing duplicate users
df3 = df3.drop_duplicates()
calculateMissing(df3)

<h1>Querying using sql</h1>

In [ ]:
# creating a querying function.
queryDf = lambda q: sqldf(q, globals())


In [ ]:
def getTopHandsets(num):
    query = 'SELECT "Handset Type", count(*) as user_count FROM df3 WHERE "Handset Type" != "undefined" group by "Handset Type" order by user_count DESC LIMIT '+str(num)
    return queryDf(query)

def get_top_n(df, colname, num, globalDict):
    queryDf = lambda q: sqldf(q, globalDict)
    query = 'SELECT "'+colname+'", count(*) as user_count FROM '+df+' WHERE "'+colname+'" != "undefined" group by "'+colname+'" order by user_count DESC LIMIT '+str(num)
    return queryDf(query)

print(get_top_n('df','Handset Type',10,globals() ))

In [ ]:
def getTopManu(num):
    query = 'SELECT "Handset Manufacturer", count(*) as user_count FROM df3 WHERE "Handset Type" != "undefined" group by "Handset Manufacturer" order by user_count DESC LIMIT '+str(num)
    return queryDf(query)
print(getTopManu(3))

In [ ]:
def manByHandset(lis, dfname):
    for man in lis:
        query = 'SELECT "Handset Manufacturer", "Handset Type", count(*) as num_users \
        FROM '+dfname+'\
        WHERE "Handset Manufacturer" = "'+man+'" \
        group by "Handset Type" \
        order by num_users DESC \
        LIMIT 3'
        print(queryDf(query),'\n')
        


manByHandset(getTopManu(5)["Handset Manufacturer"].to_list(),'df3')

In [ ]:
def find_agg(df, group_columns, agg_columns, agg_metrics, new_columns):
    new_column_dict ={}
    agg_dict = {}
    for i in range(len(agg_columns)):
        new_column_dict[agg_columns[i]] = new_columns[i]
        agg_dict[agg_columns[i]] = agg_metrics[i]

    new_df = df.groupby(group_columns).agg(agg_dict).reset_index().rename(columns=new_column_dict)
    return new_df
def combineColumns(df, col1, col2, new_name, rem=False):
    df[new_name] = df[col1]+df[col2]
    if(rem):
        df.drop([col1, col2], axis = 1, inplace = True)

grouping_lis = ["MSISDN/Number"]
aggr_lis = ["Bearer Id", "Dur. (ms)", "Total DL (Bytes)", "Total UL (Bytes)", "Social Media DL (Bytes)", "Social Media UL (Bytes)",\
    "Youtube DL (Bytes)", "Youtube UL (Bytes)", "Netflix DL (Bytes)", "Netflix UL (Bytes)", "Google DL (Bytes)", "Google UL (Bytes)", "Email DL (Bytes)", "Email UL (Bytes)", "Other DL (Bytes)", "Other UL (Bytes)"] 
metric_lis = ["count", "sum", "sum", "sum", "sum", "sum", "sum", "sum","sum", "sum", "sum", "sum", "sum", "sum", "sum","sum"]
col_names = ["xDr_session_count", "session_dur", "Total_DL_sum", "Total_UL_sum", "Social_Media_DL_sum", "Social_Media_UL_sum",\
    "Youtube_DL_total", "Youtube_UL_total", "Netflix_DL_total", "Netflix_UL_total", "Google_DL_total", "Google_UL_total", "Email_DL_total", "Email_UL_total", "Other_DL_total", "Other_UL_total"]


aggr_df = find_agg(df, grouping_lis, aggr_lis, metric_lis, col_names)
combineULDL(aggr_df, "Youtube_DL_total", "Youtube_UL_total")
combineULDL(aggr_df, "Netflix_DL_total", "Netflix_UL_total")
combineULDL(aggr_df, "Google_DL_total", "Google_UL_total")
combineULDL(aggr_df, "Email_DL_total", "Email_UL_total")
combineULDL(aggr_df, "Social_Media_DL_sum", "Social_Media_UL_sum")
combineULDL(aggr_df, "Other_DL_total", "Other_UL_total")

calculateMissing(aggr_df)

aggr_df

In [ ]:
def format_number(df, cols):
    float_format_list = []

    for i in range(cols.shape[0]):
        if(cols.iloc[i,3] == "float64"):
            float_format_list.append(cols.iloc[i,0])
    for col in float_format_list:
        df[col] = df.apply(lambda row : f'{row[col]:,.2f}', axis = 1)
    
    return df
    


In [ ]:
def convert_bytes_to_megabytes(df, cols):
    bytes_list = []
    megabyte = 1*10e+5
    
    for i in range(cols.shape[0]):
        if("(Bytes)" in cols.iloc[i,0]):
            bytes_list.append(cols.iloc[i,0])
    
    for col in bytes_list:
        df[col] = df[col]/megabyte
        df.rename(columns={col:col.replace('(Bytes)','(MB)')}, inplace=True)
    return df

In [ ]:
df3 = convert_bytes_to_megabytes(df3, calculateMissing(df3))
calculateMissing(df3)


In [ ]:
aggr_df.describe()

<h2> Exploratory Data Analysis </h2>

In [ ]:
#creating frequency tables.
def generateFreqTable(df, cols):
    for col in cols:
        print(df[col].value_counts().head())
        
generateFreqTable(aggr_df, ['xDr_session_count', 'session_dur'])

In [ ]:
def summary_one(df, cols):
    df2 = df[cols]
    
    df_sum = df2.max().to_frame().reset_index().rename(columns={"index":"variables",0:"max"})
    df_sum["min"] = df2.min().to_frame().reset_index().iloc[:,1]
    df_sum['range'] = df_sum['max'] - df_sum['min']
    df_sum["count"] = df2.count().to_frame().reset_index().iloc[:,1]
    return df_sum
def summary_two(df, cols):
    df2 = df[cols]
    df_sum = df2.mean().to_frame().reset_index().rename(columns={"index":"variables",0:"mean"})
    df_sum["median"] = df2.median().to_frame().reset_index().iloc[:,1]
    df_sum["mode"] = df2.mode().iloc[:,1]
    return df_sum
def summary_three(df, cols):
    df2 = df[cols]
    df_sum = df2.std().to_frame().reset_index().rename(columns={"index":"variables",0:"std"})
    df_sum["var"] = df2.var().to_frame().reset_index().iloc[:,1]
    return df_sum


summary_one(aggr_df, ['xDr_session_count', 'session_dur'])
        

In [ ]:
def dipCat(df, cols):
    plot_df = df[cols]
    plt.figure(figsize=(25, 12))
    sns.countplot(x="Handset Manufacturer", data=plot_df.head())
    
dipCat(df3, ['Handset Type', 'Handset Manufacturer'])

In [ ]:
def bivariateAnalysis(df, col , cols, crosst=False):
    for col2 in cols:
        df_plot = df[[col2, col]]
        sns.relplot(data=df_plot, aspect = 2.0)
        if(crosst):
            pd.crosstab(df[col2], df[col], margins=True)
    
        
aggr_df['Total_vol'] = aggr_df['Total_DL_sum'] + aggr_df['Total_UL_sum']
com_list = ['Netflix_Total Vol_total', 'Google_Total Vol_total']
bivariateAnalysis(aggr_df, 'Total_vol', com_list)

In [ ]:
def topDecile(df, group, cols, metric, name, top=5):
    aggr_n = find_agg(df, group, cols, metric, name)
    aggr_n = aggr_n.loc[aggr_n['Decile'] < top+1]
    return aggr_n

aggr_df['Decile'] = pd.qcut(aggr_df['session_dur'], 10, labels=False)
topDecile(aggr_df, ['Decile'], ['Total_vol'], ['sum'], ['Total_vol_p_decile'], 6 )



In [ ]:
def corrMatrix(df, cols):
    relation = df[cols].corr()
    return relation

corrMatrix(aggr_df, ['Youtube_Total Vol_total', 'Google_Total Vol_total', 'Netflix_Total Vol_total'])

In [ ]:
def low_variance_filter(df):
    df.drop(['A'], axis = 1)